In [ ]:

from tensorflow.keras.layers import Input, Conv3D, BatchNormalization, Activation, MaxPooling3D, Concatenate, Dropout, SpatialDropout3D, TimeDistributed, Dense, Bidirectional, GRU
from tensorflow.keras.models import Model

def conv_block(input_tensor, filters, kernel_size):
    x = Conv3D(filters, kernel_size, padding='same')(input_tensor)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = MaxPooling3D((2, 2, 2), strides=(2, 2, 2))(x)
    return x

def dense_block(input_tensor, num_layers, growth_rate, dropout_rate):
    x = input_tensor
    for _ in range(num_layers):
        conv = Conv3D(growth_rate, kernel_size=(3, 3, 3), padding='same')(x)
        conv = BatchNormalization()(conv)
        conv = Activation('relu')(conv)
        conv = Dropout(dropout_rate)(conv)
        x = Concatenate()([x, conv])
    return x

def transition_block(input_tensor, compression_factor, dropout_rate):
    x = Conv3D(int(input_tensor.shape[-1] * compression_factor), kernel_size=(1, 1, 1), padding='same')(input_tensor)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Dropout(dropout_rate)(x)
    x = MaxPooling3D((2, 2, 2), strides=(2, 2, 2))(x)
    return x

def parallel_branches(input_tensor, filters, kernel_size, dropout_rate):
    branch1 = Conv3D(filters, kernel_size, padding='same')(input_tensor)
    branch1 = BatchNormalization()(branch1)
    branch1 = Activation('relu')(branch1)

    branch2 = Conv3D(filters, kernel_size, padding='same')(input_tensor)
    branch2 = BatchNormalization()(branch2)
    branch2 = Activation('relu')(branch2)
    branch2 = Dropout(dropout_rate)(branch2)

    branch3 = Conv3D(filters, kernel_size, padding='same')(input_tensor)
    branch3 = BatchNormalization()(branch3)
    branch3 = Activation('relu')(branch3)
    branch3 = SpatialDropout3D(dropout_rate)(branch3)

    merged = Concatenate()([branch1, branch2, branch3])
    return merged

# Input tensor shape
input_shape = (None, 64, 64, 1)  # (batch_size, frames, height, width, channels)
input_tensor = Input(shape=input_shape)

# First convolutional block
x = conv_block(input_tensor, filters=64, kernel_size=(7, 7, 7))

# Dense Blocks and Transition Blocks
x = dense_block(x, num_layers=6, growth_rate=32, dropout_rate=0.2)
x = transition_block(x, compression_factor=0.5, dropout_rate=0.2)
x = dense_block(x, num_layers=12, growth_rate=32, dropout_rate=0.2)
x = transition_block(x, compression_factor=0.5, dropout_rate=0.2)
x = dense_block(x, num_layers=24, growth_rate=32, dropout_rate=0.2)
x = transition_block(x, compression_factor=0.5, dropout_rate=0.2)
x = dense_block(x, num_layers=16, growth_rate=32, dropout_rate=0.2)

# Parallel processing layer
x = parallel_branches(x, filters=128, kernel_size=(3, 3, 3), dropout_rate=0.2)

# Reshape the tensor to (batch_size, timesteps, features)
timesteps = tf.shape(x)[1]
features = x.shape[2] * x.shape[3] * x.shape[4]
x = tf.reshape(x, (-1, timesteps, features))


# Add a TimeDistributed wrapper around a Dense layer
x = TimeDistributed(Dense(units=256))(x)

# Add two Bidirectional GRU layers
x = Bidirectional(GRU(units=128, return_sequences=True))(x)
x = Bidirectional(GRU(units=128, return_sequences=True))(x)

# Add a Dense layer
num_classes = char_to_num.vocabulary_size()  # Number of output classes based on the vocabulary size
x = Dense(units=num_classes)(x)

# Add a Softmax activation layer
output = Activation('softmax')(x)

# Create the model
model = Model(inputs=input_tensor, outputs=output)

print("Model output shape:", model.output_shape)
print("Expected label shape:", (None, None))  # None for batch size and timesteps



Model output shape: (None, None, 40)
Expected label shape: (None, None)


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences

def pad_labels(video, alignment):
    max_length = tf.shape(alignment)[1]
    padded_alignment = tf.pad(alignment, [[0, 0], [0, max_length - tf.shape(alignment)[1]]], constant_values=-1)
    return video, padded_alignment

train_dataset = train_dataset.map(pad_labels)
val_dataset = val_dataset.map(pad_labels)

# Create the mask for true labels
def create_mask(video, alignment):
    mask = tf.cast(tf.not_equal(alignment, -1), tf.float32)
    return video, alignment, mask

train_dataset = train_dataset.map(create_mask)
val_dataset = val_dataset.map(create_mask)

# Modify the CTC loss function
def ctc_loss(y_true, y_pred):
    label_length = tf.reduce_sum(tf.cast(tf.not_equal(y_true, -1), tf.int32), axis=-1)
    input_length = tf.shape(y_pred)[1] * tf.ones(shape=tf.shape(label_length), dtype=tf.int32)
    loss = tf.nn.ctc_loss(
        labels=y_true,
        logits=y_pred,
        label_length=label_length,
        logit_length=input_length,
        logits_time_major=False,
        blank_index=-1
    )
    return tf.reduce_mean(loss)

# Compile the model
model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss=ctc_loss,
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy()]
)

# Train the model
history = model.fit(
    train_dataset,
    epochs=2,
    validation_data=val_dataset
)


Epoch 1/2


Instructions for updating:
Prefer tf.tensor_scatter_nd_add, which offers the same functionality with well-defined read-write semantics.
Instructions for updating:
Prefer tf.tensor_scatter_nd_update, which offers the same functionality with well-defined read-write semantics.


ResourceExhaustedError: Graph execution error:

Detected at node model_2/max_pooling3d_8/MaxPool3D defined at (most recent call last):
  File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main

  File "/usr/lib/python3.10/runpy.py", line 86, in _run_code

  File "/usr/local/lib/python3.10/dist-packages/colab_kernel_launcher.py", line 37, in <module>

  File "/usr/local/lib/python3.10/dist-packages/traitlets/config/application.py", line 992, in launch_instance

  File "/usr/local/lib/python3.10/dist-packages/ipykernel/kernelapp.py", line 619, in start

  File "/usr/local/lib/python3.10/dist-packages/tornado/platform/asyncio.py", line 195, in start

  File "/usr/lib/python3.10/asyncio/base_events.py", line 603, in run_forever

  File "/usr/lib/python3.10/asyncio/base_events.py", line 1909, in _run_once

  File "/usr/lib/python3.10/asyncio/events.py", line 80, in _run

  File "/usr/local/lib/python3.10/dist-packages/tornado/ioloop.py", line 685, in <lambda>

  File "/usr/loc